In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv("../data/pvlive_2016_2023.csv")

In [ ]:
display(df)

In [ ]:
px.scatter(df, x="start_datetime_utc",y="capacity_mwp")

In [ ]:
px.scatter(df, x="start_datetime_utc",y="installedcapacity_mwp")

In [ ]:
# Get peak generation per day
df['date'] = pd.to_datetime(df['start_datetime_utc']).dt.date
daily_peak = df.groupby('date').agg({
    'generation_mw': 'max',
    'capacity_mwp': 'first'
}).reset_index()

# Create scatter plot of daily peak generation vs capacity
fig = px.scatter(
    daily_peak,
    x="capacity_mwp", 
    y="generation_mw",
    title="Daily Peak Solar Generation vs Capacity",
    labels={
        "capacity_mwp": "Capacity (MWp)",
        "generation_mw": "Peak Generation (MW)",
        "date": "Date"
    },
    hover_data=["date"]
)
fig.show()
